In [23]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.metrics import mean_absolute_error

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# === Load Dataset ===
DATA = Path("New_approach/dataset/cleaned_datasets/all_cities_weather_ready_train.parquet")
df = pd.read_parquet(DATA, engine="fastparquet")

# === Define Features ===
TARGET = "kWh_per_m2"
CAT = ["BuildingType"]
NUM = [
    "tilt", "tilt2", "tilt_sin", "tilt_cos",
    "GHI_kWh_per_m2_day", "AvgTemp_C",
    "ClearnessIndex", "Precip_mm_per_day"
]

X = df[NUM + CAT]
y = df[TARGET]

# ✅ Convert categorical columns to 'category' dtype
for col in CAT:
    X[col] = X[col].astype("category")


C:\Users\Admin\AppData\Local\Temp\ipykernel_13084\529137900.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype("category")


Cell 2 — Fine-Tuning LightGBM

In [29]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor, early_stopping
from xgboost import XGBRegressor

# === Transform target ===
y_log = np.log1p(y)

# === Prepare separate data for XGBoost (encode categorical) ===
X_xgb = X.copy()
X_xgb["BuildingType"] = X_xgb["BuildingType"].cat.codes

# === Base models ===
xgb = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1500,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    enable_categorical=False  # we encoded categories manually
)

lgb = LGBMRegressor(
    objective='mae',
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=15,
    min_child_samples=100,
    lambda_l1=1.0,
    lambda_l2=1.0,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# === Cross-validation and meta stacking ===
cv = GroupKFold(n_splits=3)
mae_scores = []

for fold, (tr, va) in enumerate(cv.split(X, y_log, groups=df["City"]), 1):
    # Use encoded data for XGB and original (categorical) for LGB
    X_tr_xgb, X_va_xgb = X_xgb.iloc[tr], X_xgb.iloc[va]
    X_tr_lgb, X_va_lgb = X.iloc[tr], X.iloc[va]
    y_tr, y_va = y_log.iloc[tr], y_log.iloc[va]

    # Train XGBoost
    xgb.fit(X_tr_xgb, y_tr, eval_set=[(X_va_xgb, y_va)], verbose=False)

    # Train LightGBM with early stopping (new callback style)
    lgb.fit(
        X_tr_lgb, y_tr,
        eval_set=[(X_va_lgb, y_va)],
        callbacks=[early_stopping(stopping_rounds=150, verbose=False)]
    )

    # Predictions (inverse log1p transform)
    pred_xgb = np.expm1(xgb.predict(X_va_xgb))
    pred_lgb = np.expm1(lgb.predict(X_va_lgb))

    # --- Blend weights: test 0.3–0.7 LGB fraction ---
    blended = 0.3 * pred_xgb + 0.7 * pred_lgb

    # --- Meta-learner (stacking) ---
    meta = Ridge(alpha=1.0)
    meta.fit(np.column_stack([pred_xgb, pred_lgb]), np.expm1(y_va))
    stacked = meta.predict(np.column_stack([pred_xgb, pred_lgb]))

    mae = mean_absolute_error(np.expm1(y_va), stacked)
    mae_scores.append(mae)
    print(f"Fold {fold} MAE = {mae:.3f}")

print(f"\n🎯 Mean Stacked Ensemble MAE: {np.mean(mae_scores):.3f} ± {np.std(mae_scores):.3f}")


[LightGBM] [Warning] lambda_l1 is set=1.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] lambda_l1 is set=1.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024858 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1103
[LightGBM] [Info] Number of data points in the train set: 4157956, number of used features: 9
[LightGBM] [Warning] lambda_l1 is set=1.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Info] Start t

Cell 3 — XGBoost + LightGBM Blending (Ensemble)

In [13]:
# from lightgbm import early_stopping
# from pathlib import Path

# from sklearn.model_selection import GroupKFold, GridSearchCV
# from sklearn.metrics import mean_absolute_error

# from lightgbm import LGBMRegressor, early_stopping
# from xgboost import XGBRegressor

# # --- Convert categorical column for both frameworks ---
# X["BuildingType"] = X["BuildingType"].astype("category")
# X_xgb = X.copy()
# X_xgb["BuildingType"] = X_xgb["BuildingType"].cat.codes

# # === Define Models ===
# xgb = XGBRegressor(
#     objective="reg:squarederror",
#     n_estimators=800,
#     learning_rate=0.05,
#     max_depth=6,
#     subsample=0.9,
#     colsample_bytree=0.8,
#     random_state=42,
#     enable_categorical=False  # using integer-encoded categories
# )

# lgb = LGBMRegressor(
#     objective="mae",
#     n_estimators=1000,
#     learning_rate=0.05,
#     num_leaves=31,
#     subsample=0.9,
#     colsample_bytree=0.8,
#     random_state=42
# )

# # === Cross-Validation ===
# cv = GroupKFold(n_splits=3)
# mae_scores = []

# for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups=df["City"]), 1):
#     X_train_xgb, X_val_xgb = X_xgb.iloc[train_idx], X_xgb.iloc[val_idx]
#     X_train_lgb, X_val_lgb = X.iloc[train_idx], X.iloc[val_idx]
#     y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     # Train XGBoost
#     xgb.fit(X_train_xgb, y_train, eval_set=[(X_val_xgb, y_val)], verbose=False)

#     # Train LightGBM with early stopping via callback
#     lgb.fit(
#         X_train_lgb, y_train,
#         eval_set=[(X_val_lgb, y_val)],
#         callbacks=[early_stopping(stopping_rounds=100, verbose=False)]
#     )

#     pred_xgb = xgb.predict(X_val_xgb)
#     pred_lgb = lgb.predict(X_val_lgb)

#     # Weighted blend (tune ratio if needed)
#     blended = 0.6 * pred_xgb + 0.4 * pred_lgb
#     mae = mean_absolute_error(y_val, blended)
#     mae_scores.append(mae)
#     print(f"Fold {fold} MAE = {mae:.3f}")

# print(f"\n✅ Mean Ensemble MAE: {np.mean(mae_scores):.3f} ± {np.std(mae_scores):.3f}")


C:\Users\Admin\AppData\Local\Temp\ipykernel_13084\2055137006.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["BuildingType"] = X["BuildingType"].astype("category")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026595 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1103
[LightGBM] [Info] Number of data points in the train set: 4157956, number of used features: 9
[LightGBM] [Info] Start training from score 275.880188
Fold 1 MAE = 14.747
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017955 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1092
[LightGBM] [Info] Number of data points in the train set: 4163079, number of used features: 9
[LightGBM] [Info] Start training from score 257.323975
Fold 2 MAE = 15.263
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023377 seconds.
You can set `force_row_wise=true` to r

In [31]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor, early_stopping
from xgboost import XGBRegressor

cv = GroupKFold(n_splits=3)

# === Slightly relaxed LightGBM for finer detail ===
lgb_params = dict(
    objective='mae',
    n_estimators=2500,
    learning_rate=0.025,
    num_leaves=31,           # up from 15
    min_child_samples=50,    # was 100
    lambda_l1=0.5,
    lambda_l2=0.5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_params = dict(
    objective='reg:squarederror',
    n_estimators=1500,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# === Transform target (log-scale) ===
y_log = np.log1p(y)

results = []

# --- Prepare encoded copy for XGB (convert categorical to numeric codes) ---
X_xgb = X.copy()
for col in X_xgb.select_dtypes(include=["category"]).columns:
    X_xgb[col] = X_xgb[col].cat.codes

# === Run blending and stacking search ===
for lgb_w in [0.5, 0.6, 0.7]:
    for alpha in [0.1, 0.5, 1.0]:
        fold_mae = []
        for fold, (tr, va) in enumerate(cv.split(X, y_log, groups=df["City"]), 1):
            X_tr_lgb, X_va_lgb = X.iloc[tr], X.iloc[va]
            X_tr_xgb, X_va_xgb = X_xgb.iloc[tr], X_xgb.iloc[va]
            y_tr, y_va = y_log.iloc[tr], y_log.iloc[va]

            xgb = XGBRegressor(**xgb_params)
            lgb = LGBMRegressor(**lgb_params)

            # XGB uses numeric-encoded data
            xgb.fit(X_tr_xgb, y_tr, eval_set=[(X_va_xgb, y_va)], verbose=False)

            # LGBM uses native categorical data with new callback style
            lgb.fit(
                X_tr_lgb, y_tr,
                eval_set=[(X_va_lgb, y_va)],
                callbacks=[early_stopping(stopping_rounds=150, verbose=False)]
            )

            # Predictions (back-transform from log1p)
            pred_xgb = np.expm1(xgb.predict(X_va_xgb))
            pred_lgb = np.expm1(lgb.predict(X_va_lgb))

            # Blended prediction
            blended = (1 - lgb_w) * pred_xgb + lgb_w * pred_lgb

            # Meta learner (Ridge stacking)
            meta = Ridge(alpha=alpha)
            meta.fit(np.column_stack([pred_xgb, pred_lgb]), np.expm1(y_va))
            stacked = meta.predict(np.column_stack([pred_xgb, pred_lgb]))

            mae = mean_absolute_error(np.expm1(y_va), stacked)
            fold_mae.append(mae)

            print(f"Fold {fold} → MAE={mae:.3f}")

        mean_mae, std_mae = np.mean(fold_mae), np.std(fold_mae)
        results.append((lgb_w, alpha, mean_mae, std_mae))
        print(f"w={lgb_w}, alpha={alpha} → MAE={mean_mae:.3f} ± {std_mae:.3f}")

best_combo = min(results, key=lambda x: x[2])
print("\n🏆 Best combo → "
      f"LGB_weight={best_combo[0]}, alpha={best_combo[1]}, "
      f"MAE={best_combo[2]:.3f} ± {best_combo[3]:.3f}")


[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024686 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1103
[LightGBM] [Info] Number of data points in the train set: 4157956, number of used features: 9
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Info] Start t

In [15]:
best_lgbm_mae = -grid.best_score_
ensemble_mae = np.mean(mae_scores)

print(f"📊 Baseline LGBM (from previous run): 17.23 kWh/m²")
print(f"✅ Tuned LGBM MAE: {best_lgbm_mae:.3f} kWh/m²")
print(f"🤝 Ensemble MAE: {ensemble_mae:.3f} kWh/m²")
print(f"📉 Improvement: {17.23 - ensemble_mae:.2f} kWh/m² (~{(17.23 - ensemble_mae)/17.23*100:.1f}%)")


NameError: name 'grid' is not defined